In [4]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [5]:
Predicted = pd.read_csv("./Stereoscope/Outputs/output_DestVISim.csv")
Target=pd.read_csv("./Data/DestVISim/target.csv")

In [6]:
Predicted

,Unnamed: 0,0,1,2,3,4
0,0,0.045357,4.636598e-01,2.754686e-01,0.116282,0.099232
1,1,0.042728,6.006261e-01,2.250844e-01,0.058889,0.072673
2,2,0.059723,6.360124e-01,2.055784e-01,0.029930,0.068756
3,3,0.064960,6.741626e-01,6.807286e-02,0.066495,0.126310
4,4,0.138686,6.461712e-01,1.381660e-02,0.032541,0.168785
...,...,...,...,...,...,...
1595,1595,0.118818,2.224836e-01,2.301819e-02,0.040637,0.595043
1596,1596,0.240801,1.785520e-01,2.303550e-06,0.113962,0.466683
1597,1597,0.275864,3.670646e-02,2.319032e-07,0.032871,0.654559
1598,1598,0.391388,1.139364e-01,2.645098e-07,0.045622,0.449053


In [7]:
Target

,Unnamed: 0,0,1,2,3,4
0,0,0.109382,0.458548,0.240255,0.117889,0.073926
1,1,0.106379,0.542767,0.192066,0.080376,0.078412
2,2,0.115552,0.584335,0.148478,0.063775,0.087860
3,3,0.134864,0.583714,0.119500,0.059645,0.102278
4,4,0.156835,0.555801,0.104024,0.063044,0.120296
...,...,...,...,...,...,...
1595,1595,0.270532,0.229576,0.079589,0.065887,0.354416
1596,1596,0.331026,0.218634,0.057692,0.046939,0.345709
1597,1597,0.401695,0.196309,0.046337,0.036350,0.319309
1598,1598,0.478614,0.162905,0.039004,0.030636,0.288841


In [8]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)


In [9]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [10]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [11]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [12]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [13]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [14]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [15]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [16]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [17]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [18]:
np.mean(np.nan_to_num(d))

0.9315595992711614

In [19]:
d

array([0.95429647, 0.87230371, 0.95213622, 0.94594021, 0.93312138])

## Jenson Shannon Divergence

In [20]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.13126539000970186

## F1 Score, Precision, Recall

In [21]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [22]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [23]:
d

,Precision,Recall,F1-score
0.100,0.901893,0.884608,0.893167
0.200,0.851924,0.874253,0.862944
0.300,0.809717,0.883978,0.845219
0.010,0.994765,0.914051,0.952701
0.001,1.000000,0.926375,0.961781


In [ ]:
Prediction